In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='0'

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
import tensorflow as tf
import numpy as np
import numpy.random as rng
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
#Creating the dataset 
inshape=784 #input size
hiddenshape=1000 # No of hidden units 
outshape=10 # Output size 

batch_size = 50

#Training parameters 
iterations=3000
i=tf.Variable(0,name='loop_i')
eta=tf.constant(0.00001,dtype=tf.float64)

#Training data inputs
'''x=tf.placeholder(tf.float64,[None,inshape], name = 'x')
y=tf.placeholder(tf.float64,[None,outshape], name = 'y')
'''
x=tf.placeholder(tf.float64,[batch_size,784])
y=tf.placeholder(tf.float64,[batch_size,10])

#Linear network
#B=tf.random_normal([hiddenshape,outshape],mean=0.0,stddev=1.0,dtype=tf.float64)
#w_1=tf.random_normal([inshape,hiddenshape],mean=0.0,stddev=1.0,dtype=tf.float64)
#w_2=tf.random_normal([hiddenshape,outshape],mean=0.0,stddev=1.0,dtype=tf.float64)

#parameters for feedback alignment
B = tf.Variable(rng.randn(hiddenshape, outshape))
w_1 = tf.Variable(rng.randn(hiddenshape, inshape))
w_2= tf.Variable(rng.randn(outshape, hiddenshape))

hidout=tf.matmul(w_1,x)
y_pred_fa=tf.matmul(w_2,hidout) 

e_fa=y_pred_fa-y
cost_fa = tf.reduce_sum(tf.pow(e_fa, 2))/2
Be=tf.matmul(B,e_fa)

dLw1=tf.matmul(Be,tf.transpose(x))
dLw2=tf.matmul(e_fa,tf.transpose(hidout))

#Parameters for back propogation

w1bp=tf.Variable(hiddenshape,outshape)
w2bp=tf.Variable(outshape,hiddenshape)

w1bp=w_1 
w2bp=w_2

hidout_bp=tf.matmul(w1bp,x)
y_pred_bp=tf.matmul(w2bp,hidout_bp)

e_bp=y_pred_bp-y
cost_bp = tf.reduce_sum(tf.pow(e_bp, 2))/2
We=tf.matmul(tf.transpose(w2bp),e_bp)

dLw1bp=tf.matmul(We,tf.transpose(x))
dLw2bp=tf.matmul(e_bp,tf.transpose(hidout_bp))

#Training data generation
#x=tf.random_normal([inshape,1],mean=0.0,stddev=1.0,dtype=tf.float64)
#y=tf.matmul(T,x)
#T=tf.random_normal([outshape,inshape],mean=0.0,stddev=1.0,dtype=tf.float64)
# T = rng.randn(outshape, inshape)
#def traindata(T, batch_size):
 #   train_x = rng.randn(T.shape[1], batch_size)
  #  train_y = np.dot(T, train_x)
   # return (train_x, train_y)''''

ValueError: Dimensions must be equal, but are 784 and 50 for 'MatMul' (op: 'MatMul') with input shapes: [1000,784], [50,784].

In [ ]:
def updatefa(w_1,w_2):    
    #gradients for a loss L 
    #Update
    #w_1=tf.subtract(w_1,tf.multiply(eta,tf.transpose(dLw1)))
    #w_2=tf.subtract(w_2,tf.multiply(eta,tf.transpose(dLw2))) 
    new_w1 = w_1.assign(w_1-tf.multiply(eta,dLw1))
    new_w2 = w_2.assign(w_2-tf.multiply(eta,dLw2)) 
    return new_w1, new_w2 

def updatebp(w1bp,w2bp):
    new_w1bp=w1bp.assign(w1bp-tf.multiply(eta,dLw1bp))
    new_w2bp=w2bp.assign(w2bp-tf.multiply(eta,dLw2bp))
    return new_w1bp,new_w2bp


In [ ]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for idx in range(iterations):
        (train_x, train_y) = mnist.train.next_batch(batch_size)
        #sess.run(update(w_1, w_2), feed_dict = {x:train_x, y:train_y})
        #for jj in range(batch_size):
        sess.run(updatefa(w_1,w_2), feed_dict={x: train_x, y: train_y})
        sess.run(updatebp(w1bp,w2bp),feed_dict={x: train_x, y: train_y})
        cfa = sess.run(cost_fa, feed_dict={x: train_x, y: train_y})
        cbp = sess.run(cost_bp, feed_dict={x: train_x, y: train_y})
        if idx % 50==0:
            cfa = sess.run(cost_fa, feed_dict={x: train_x, y: train_y})
            cbp = sess.run(cost_bp, feed_dict={x: train_x, y: train_y})
            print("iteration: %d Feedbackalignment: %f\tBackprop:%f"%(idx,cfa,cbp))
            

In [ ]:
x=tf.placeholder(tf.float64,[None,inshape], name = 'x')
y=tf.placeholder(tf.float64,[None,outshape])
sess=tf.Session()
sess.run(updatefa(w_1, w_2), feed_dict = {x:train_x, y:train_y})

In [ ]:
#x=tf.placeholder(tf.float64,[None,784])
#y=tf.placeholder(tf.float64,[None,10])

x=tf.placeholder(tf.float64,[batch_size,784])
y=tf.placeholder(tf.float64,[batch_size,10])
x:train_x
y:train_y
np.shape(train_y)


In [ ]:
batch_size